In [ ]:
from collections import Counter

import nltk
from ipynb.fs.full.log_entries import get_all_log_entries
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

%matplotlib inline
nltk.download('punkt')

In [ ]:
def timepot_thrown_exception_counter():
    filename_prefix = 'timepot'
    exception_java_class_counter = Counter()
    entries = get_all_log_entries(filename_prefix)
    for entry in entries:
        tokenized_msg = nltk.word_tokenize(entry['msg'])
        is_exception_entry = False
        for token in tokenized_msg:
            if token.endswith('Exception'):
                subtokens = token.split(sep = '.')
                exception_java_class = subtokens[-1]
                if exception_java_class[0].isupper():
                    exception_java_class_counter[exception_java_class] += 1
                    is_exception_entry = True
            if is_exception_entry is True:
                break
    return exception_java_class_counter

In [ ]:
def is_exception_entry(entry):
    tokenized_msg = nltk.word_tokenize(entry['msg'])
    for token in tokenized_msg:
        if token.endswith('Exception'):
            subtokens = token.split(sep = '.')
            exception_java_class = subtokens[-1]
            if exception_java_class[0].isupper():
                return True
    return False

def timepot_thrown_exceptions_per_day_counter():
    filename_prefix = 'timepot'
    exception_per_day_counter = Counter()
    all_entries_counter = Counter()
    entries = get_all_log_entries(filename_prefix)
    for entry in entries:
        entry_log_date = entry['log_date'].date()
        all_entries_counter[entry_log_date] += 1
        if is_exception_entry(entry):
            exception_per_day_counter[entry_log_date] += 1
    return exception_per_day_counter, all_entries_counter

In [ ]:
def plot_thrown_exception_count_distribution():
    per_day_counter, all_entries_counter = timepot_thrown_exceptions_per_day_counter()
    per_day_counter_sorted = sorted(per_day_counter.items(), key=lambda x: x[0])
    
    mat_dates = mdates.date2num([x[0] for x in per_day_counter_sorted])
    mat_values = [x[1] for x in per_day_counter_sorted]
    
    plt.figure(figsize=(10, 6))
    plt.grid(True)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=7))
    plt.plot_date(mat_dates, mat_values, '-o')

In [ ]:
def plot_thrown_exception_percentage_distribution():
    per_day_counter, all_entries_counter = timepot_thrown_exceptions_per_day_counter()
    per_day_counter_sorted = sorted(per_day_counter.items(), key=lambda x: x[0])
    all_entries_counter_sorted = sorted(all_entries_counter.items(), key=lambda x: x[0])
    
    mat_dates = mdates.date2num([x[0] for x in per_day_counter_sorted[1:]])
    mat_values = [x[1]/y[1] * 100 for x, y in zip(per_day_counter_sorted[1:], all_entries_counter_sorted[1:])]
    
    plt.figure(figsize=(10, 6))
    plt.grid(True)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=7))
    plt.plot_date(mat_dates, mat_values, '-o')